In [1]:
import pandas as pd
import numpy as np

df_annual_em_ela_2019 = pd.read_csv(
    "/Users/kashafali/Documents/Duke/Spring23/UDS/Project/Data/2019/Annual EM Math.csv"
)


In [2]:
df_annual_em_ela_2019.columns


Index(['ENTITY_CD', 'ENTITY_NAME', 'YEAR', 'ASSESSMENT_NAME', 'SUBGROUP_NAME',
       'NUM_TESTED', 'NOT_TESTED', 'LEVEL1_COUNT', 'LEVEL1_%TESTED',
       'LEVEL2_COUNT', 'LEVEL2_%TESTED', 'LEVEL3_COUNT', 'LEVEL3_%TESTED',
       'LEVEL4_COUNT', 'LEVEL4_%TESTED', 'LEVEL5_COUNT', 'LEVEL5_%TESTED',
       'NUM_PROF', 'PER_PROF', 'TOTAL_SCALE_SCORES', 'MEAN_SCORE'],
      dtype='object')

In [3]:
df_annual_em_ela_2019["SUBGROUP_NAME"].unique()

array(['Not Migrant', 'Parent in Armed Forces',
       'Parent Not in Armed Forces', 'All Students',
       'American Indian or Alaska Native',
       'Asian or Native Hawaiian/Other Pacific Islander',
       'Black or African American', 'Economically Disadvantaged',
       'English Language Learners', 'Female', 'General Education',
       'Hispanic or Latino', 'Homeless', 'Male', 'Multiracial',
       'Non-English Language Learners', 'Not Economically Disadvantaged',
       'Not Homeless', 'Not in Foster Care', 'Students with Disabilities',
       'White', 'In Foster Care', 'Migrant', 'Small Group Total'],
      dtype=object)

In [4]:
# rename Non-English Language Learners to Non-English Language Learner
df_annual_em_ela_2019["SUBGROUP_NAME"] = df_annual_em_ela_2019["SUBGROUP_NAME"].replace("Non-English Language Learners", "Non-English Language Learner")
# rename English Language Learners to English Language Learner
df_annual_em_ela_2019["SUBGROUP_NAME"] = df_annual_em_ela_2019["SUBGROUP_NAME"].replace("English Language Learners", "English Language Learner")


# Dataframe for All Students

In [5]:
# only keep columns
column_names = [
    "ENTITY_CD",
    "ENTITY_NAME",
    "YEAR",
    "ASSESSMENT_NAME",
    "SUBGROUP_NAME",
    "NOT_TESTED",
    "NUM_TESTED",
    "NUM_PROF",
    "PER_PROF",
    "TOTAL_SCALE_SCORES",
    "MEAN_SCORE",
]

df_annual_em_ela_2019_total = df_annual_em_ela_2019[column_names]
df_annual_em_ela_2019_total = df_annual_em_ela_2019_total[
    df_annual_em_ela_2019_total["SUBGROUP_NAME"] == "All Students"
]


In [6]:
df_annual_em_ela_2019_total.head()


,ENTITY_CD,ENTITY_NAME,YEAR,ASSESSMENT_NAME,SUBGROUP_NAME,NOT_TESTED,NUM_TESTED,NUM_PROF,PER_PROF,TOTAL_SCALE_SCORES,MEAN_SCORE
12,1,NYC Public Schools,2018,Combined8Math,All Students,21909,64087,27892,44,00,00
13,1,NYC Public Schools,2018,Combined7Math,All Students,6675,65892,26320,40,00,00
14,1,NYC Public Schools,2018,RegentsMath8,All Students,s,14361,11369,79,NaN,NaN
15,1,NYC Public Schools,2018,MATH3_8,All Students,60289,402824,177196,44,00,00
16,1,NYC Public Schools,2018,MATH8,All Students,21909,49726,16523,33,29864407,601


In [7]:
df_annual_em_ela_2019_merge = df_annual_em_ela_2019_total.drop(
    columns=["SUBGROUP_NAME"]
).copy()


# Dataframe for Gender

In [8]:
df_annual_em_ela_2019_gender = df_annual_em_ela_2019.copy()

df_annual_em_ela_2019_gender = df_annual_em_ela_2019_gender[column_names]

df_annual_em_ela_2019_gender = df_annual_em_ela_2019_gender[
    (df_annual_em_ela_2019_gender["SUBGROUP_NAME"] == "Female")
    | (df_annual_em_ela_2019_gender["SUBGROUP_NAME"] == "Male")
]


In [9]:
# fix index
df_annual_em_ela_2019_gender = df_annual_em_ela_2019_gender.set_index(
    ["SUBGROUP_NAME", "ENTITY_CD", "YEAR", "ASSESSMENT_NAME"]
).unstack(["SUBGROUP_NAME"])

df_annual_em_ela_2019_gender = df_annual_em_ela_2019_gender.reindex(
    columns=sorted(df_annual_em_ela_2019_gender.columns, key=lambda x: x[::-1])
)

df_annual_em_ela_2019_gender.columns = [
    "{}_{}".format(t, v) for v, t in df_annual_em_ela_2019_gender.columns
]

df_annual_em_ela_2019_gender = pd.DataFrame(df_annual_em_ela_2019_gender.to_records())


In [10]:
df_annual_em_ela_2019_gender.head()


,ENTITY_CD,YEAR,ASSESSMENT_NAME,Female_ENTITY_NAME,Female_MEAN_SCORE,Female_NOT_TESTED,Female_NUM_PROF,Female_NUM_TESTED,Female_PER_PROF,Female_TOTAL_SCALE_SCORES,Male_ENTITY_NAME,Male_MEAN_SCORE,Male_NOT_TESTED,Male_NUM_PROF,Male_NUM_TESTED,Male_PER_PROF,Male_TOTAL_SCALE_SCORES
0,1,2018,MATH3,NYC Public Schools,600,3428,17599,33458.0,53,20062147,NYC Public Schools,599,5492,18269,35215.0,52,21099829
1,1,2018,MATH4,NYC Public Schools,600,3233,15570,33652.0,46,20181976,NYC Public Schools,599,4917,16354,35191.0,46,21092785
2,1,2018,MATH5,NYC Public Schools,600,3190,14149,34011.0,42,20392492,NYC Public Schools,599,4810,14767,35363.0,42,21176660
3,1,2018,MATH6,NYC Public Schools,599,2689,13067,32269.0,40,19329133,NYC Public Schools,598,3946,13209,33686.0,39,20133061
4,1,2018,MATH7,NYC Public Schools,601,2774,13363,31699.0,42,19039728,NYC Public Schools,598,3901,12790,34015.0,38,20330979


# Merge Gender Dataframe with All Students Dataframe

In [11]:
df_ela_2019_1 = pd.merge(
    df_annual_em_ela_2019_merge,
    df_annual_em_ela_2019_gender,
    on=["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],
    how="left",
)

df_ela_2019_1.head()


,ENTITY_CD,ENTITY_NAME,YEAR,ASSESSMENT_NAME,NOT_TESTED,NUM_TESTED,NUM_PROF,PER_PROF,TOTAL_SCALE_SCORES,MEAN_SCORE,...,Female_NUM_TESTED,Female_PER_PROF,Female_TOTAL_SCALE_SCORES,Male_ENTITY_NAME,Male_MEAN_SCORE,Male_NOT_TESTED,Male_NUM_PROF,Male_NUM_TESTED,Male_PER_PROF,Male_TOTAL_SCALE_SCORES
0,1,NYC Public Schools,2018,Combined8Math,21909,64087,27892,44,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NYC Public Schools,2018,Combined7Math,6675,65892,26320,40,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NYC Public Schools,2018,RegentsMath8,s,14361,11369,79,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,NYC Public Schools,2018,MATH3_8,60289,402824,177196,44,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,NYC Public Schools,2018,MATH8,21909,49726,16523,33,29864407,601,...,23701.0,36,14277816,NYC Public Schools,599,11301,8017,26025.0,31,15586591


# Dataframe for Race

In [12]:
df_annual_em_ela_2019_race = df_annual_em_ela_2019[column_names].copy()

df_annual_em_ela_2019_race = df_annual_em_ela_2019_race[
    (df_annual_em_ela_2019_race["SUBGROUP_NAME"] == "American Indian or Alaska Native")
    | (df_annual_em_ela_2019_race["SUBGROUP_NAME"] == "Black or African American")
    | (df_annual_em_ela_2019_race["SUBGROUP_NAME"] == "Hispanic or Latino")
    | (
        df_annual_em_ela_2019_race["SUBGROUP_NAME"]
        == "Asian or Native Hawaiian/Other Pacific Islander"
    )
    | (df_annual_em_ela_2019_race["SUBGROUP_NAME"] == "White")
    | (df_annual_em_ela_2019_race["SUBGROUP_NAME"] == "Multiracial")
]


In [13]:
# reindex

df_annual_em_ela_2019_race = df_annual_em_ela_2019_race.set_index(
    ["SUBGROUP_NAME", "ENTITY_CD", "YEAR", "ASSESSMENT_NAME"]
).unstack(["SUBGROUP_NAME"])

df_annual_em_ela_2019_race = df_annual_em_ela_2019_race.reindex(
    columns=sorted(df_annual_em_ela_2019_race.columns, key=lambda x: x[::-1])
)

df_annual_em_ela_2019_race.columns = [
    "{}_{}".format(t, v) for v, t in df_annual_em_ela_2019_race.columns
]

df_annual_em_ela_2019_race = pd.DataFrame(df_annual_em_ela_2019_race.to_records())


In [14]:
df_annual_em_ela_2019_race.head()


,ENTITY_CD,YEAR,ASSESSMENT_NAME,American Indian or Alaska Native_ENTITY_NAME,American Indian or Alaska Native_MEAN_SCORE,American Indian or Alaska Native_NOT_TESTED,American Indian or Alaska Native_NUM_PROF,American Indian or Alaska Native_NUM_TESTED,American Indian or Alaska Native_PER_PROF,American Indian or Alaska Native_TOTAL_SCALE_SCORES,...,Multiracial_NUM_TESTED,Multiracial_PER_PROF,Multiracial_TOTAL_SCALE_SCORES,White_ENTITY_NAME,White_MEAN_SCORE,White_NOT_TESTED,White_NUM_PROF,White_NUM_TESTED,White_PER_PROF,White_TOTAL_SCALE_SCORES
0,1,2018,MATH3,NYC Public Schools,598,82,384,771.0,50,460791,...,945.0,71,574367,NYC Public Schools,608,3196,8260,11639.0,71,7074195
1,1,2018,MATH4,NYC Public Schools,598,67,322,742.0,43,443879,...,824.0,71,503134,NYC Public Schools,609,3056,7556,11210.0,67,6824787
2,1,2018,MATH5,NYC Public Schools,596,70,266,782.0,34,465909,...,795.0,65,485755,NYC Public Schools,609,3048,7022,11206.0,63,6819549
3,1,2018,MATH6,NYC Public Schools,595,69,262,722.0,36,429931,...,641.0,68,391660,NYC Public Schools,608,2511,6583,10547.0,62,6416416
4,1,2018,MATH7,NYC Public Schools,596,69,256,745.0,34,443838,...,619.0,69,379156,NYC Public Schools,610,2328,6612,10462.0,63,6377933


# Merge Race Dataset with All Students + Gender Dataset

In [15]:
df_ela_2019_2 = pd.merge(
    df_ela_2019_1,
    df_annual_em_ela_2019_race,
    on=["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],
    how="left",
)

df_ela_2019_2.head()


,ENTITY_CD,ENTITY_NAME,YEAR,ASSESSMENT_NAME,NOT_TESTED,NUM_TESTED,NUM_PROF,PER_PROF,TOTAL_SCALE_SCORES,MEAN_SCORE,...,Multiracial_NUM_TESTED,Multiracial_PER_PROF,Multiracial_TOTAL_SCALE_SCORES,White_ENTITY_NAME,White_MEAN_SCORE,White_NOT_TESTED,White_NUM_PROF,White_NUM_TESTED,White_PER_PROF,White_TOTAL_SCALE_SCORES
0,1,NYC Public Schools,2018,Combined8Math,21909,64087,27892,44,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NYC Public Schools,2018,Combined7Math,6675,65892,26320,40,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NYC Public Schools,2018,RegentsMath8,s,14361,11369,79,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,NYC Public Schools,2018,MATH3_8,60289,402824,177196,44,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,NYC Public Schools,2018,MATH8,21909,49726,16523,33,29864407,601,...,315.0,50,191954,NYC Public Schools,608,5493,3567,7157.0,50,4352023


# English Language Learners Dataframe

In [16]:
df_annual_em_ela_2019_eng = df_annual_em_ela_2019[column_names].copy()


df_annual_em_ela_2019_eng = df_annual_em_ela_2019_eng[
    (df_annual_em_ela_2019_eng["SUBGROUP_NAME"] == "Non-English Language Learner")
    | (df_annual_em_ela_2019_eng["SUBGROUP_NAME"] == "English Language Learner")
]


In [17]:
# reindex

df_annual_em_ela_2019_eng = df_annual_em_ela_2019_eng.set_index(
    ["SUBGROUP_NAME", "ENTITY_CD", "YEAR", "ASSESSMENT_NAME"]
).unstack(["SUBGROUP_NAME"])

df_annual_em_ela_2019_eng = df_annual_em_ela_2019_eng.reindex(
    columns=sorted(df_annual_em_ela_2019_eng.columns, key=lambda x: x[::-1])
)

df_annual_em_ela_2019_eng.columns = [
    "{}_{}".format(t, v) for v, t in df_annual_em_ela_2019_eng.columns
]

df_annual_em_ela_2019_eng = pd.DataFrame(df_annual_em_ela_2019_eng.to_records())


In [18]:
df_annual_em_ela_2019_eng.head()


,ENTITY_CD,YEAR,ASSESSMENT_NAME,English Language Learner_ENTITY_NAME,English Language Learner_MEAN_SCORE,English Language Learner_NOT_TESTED,English Language Learner_NUM_PROF,English Language Learner_NUM_TESTED,English Language Learner_PER_PROF,English Language Learner_TOTAL_SCALE_SCORES,Non-English Language Learner_ENTITY_NAME,Non-English Language Learner_MEAN_SCORE,Non-English Language Learner_NOT_TESTED,Non-English Language Learner_NUM_PROF,Non-English Language Learner_NUM_TESTED,Non-English Language Learner_PER_PROF,Non-English Language Learner_TOTAL_SCALE_SCORES
0,1,2018,MATH3,NYC Public Schools,589,1199,3973,13229.0,30,7791052,NYC Public Schools,602,7721,31895,55444.0,58,33370924
1,1,2018,MATH4,NYC Public Schools,587,1028,2182,11199.0,19,6568387,NYC Public Schools,602,7122,29742,57644.0,52,34706374
2,1,2018,MATH5,NYC Public Schools,585,918,1293,9542.0,14,5580249,NYC Public Schools,601,7082,27623,59832.0,46,35988903
3,1,2018,MATH6,NYC Public Schools,584,839,1149,8896.0,13,5193537,NYC Public Schools,601,5796,25127,57059.0,44,34268657
4,1,2018,MATH7,NYC Public Schools,583,777,945,8215.0,12,4792056,NYC Public Schools,601,5898,25208,57499.0,44,34578651


# Merge English Language Learners Dataset with All Students + Gender + Race Dataset

In [19]:
df_ela_2019_3 = pd.merge(
    df_ela_2019_2,
    df_annual_em_ela_2019_eng,
    on=["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],
    how="left",
)

df_ela_2019_3.head()


,ENTITY_CD,ENTITY_NAME,YEAR,ASSESSMENT_NAME,NOT_TESTED,NUM_TESTED,NUM_PROF,PER_PROF,TOTAL_SCALE_SCORES,MEAN_SCORE,...,English Language Learner_NUM_TESTED,English Language Learner_PER_PROF,English Language Learner_TOTAL_SCALE_SCORES,Non-English Language Learner_ENTITY_NAME,Non-English Language Learner_MEAN_SCORE,Non-English Language Learner_NOT_TESTED,Non-English Language Learner_NUM_PROF,Non-English Language Learner_NUM_TESTED,Non-English Language Learner_PER_PROF,Non-English Language Learner_TOTAL_SCALE_SCORES
0,1,NYC Public Schools,2018,Combined8Math,21909,64087,27892,44,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NYC Public Schools,2018,Combined7Math,6675,65892,26320,40,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NYC Public Schools,2018,RegentsMath8,s,14361,11369,79,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,NYC Public Schools,2018,MATH3_8,60289,402824,177196,44,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,NYC Public Schools,2018,MATH8,21909,49726,16523,33,29864407,601,...,7242.0,13,4263327,NYC Public Schools,603,20300,15609,42484.0,37,25601080


# Dataframe for Economic Condition

In [20]:
df_annual_em_ela_2019_econ = df_annual_em_ela_2019[column_names].copy()

df_annual_em_ela_2019_econ = df_annual_em_ela_2019_econ[
    (df_annual_em_ela_2019_econ["SUBGROUP_NAME"] == "Economically Disadvantaged")
    | (df_annual_em_ela_2019_econ["SUBGROUP_NAME"] == "Not Economically Disadvantaged")
]


In [21]:
# reindex

df_annual_em_ela_2019_econ = df_annual_em_ela_2019_econ.set_index(
    ["SUBGROUP_NAME", "ENTITY_CD", "YEAR", "ASSESSMENT_NAME"]
).unstack(["SUBGROUP_NAME"])

df_annual_em_ela_2019_econ = df_annual_em_ela_2019_econ.reindex(
    columns=sorted(df_annual_em_ela_2019_econ.columns, key=lambda x: x[::-1])
)

df_annual_em_ela_2019_econ.columns = [
    "{}_{}".format(t, v) for v, t in df_annual_em_ela_2019_econ.columns
]

df_annual_em_ela_2019_econ = pd.DataFrame(df_annual_em_ela_2019_econ.to_records())


In [22]:
df_annual_em_ela_2019_econ.head()


,ENTITY_CD,YEAR,ASSESSMENT_NAME,Economically Disadvantaged_ENTITY_NAME,Economically Disadvantaged_MEAN_SCORE,Economically Disadvantaged_NOT_TESTED,Economically Disadvantaged_NUM_PROF,Economically Disadvantaged_NUM_TESTED,Economically Disadvantaged_PER_PROF,Economically Disadvantaged_TOTAL_SCALE_SCORES,Not Economically Disadvantaged_ENTITY_NAME,Not Economically Disadvantaged_MEAN_SCORE,Not Economically Disadvantaged_NOT_TESTED,Not Economically Disadvantaged_NUM_PROF,Not Economically Disadvantaged_NUM_TESTED,Not Economically Disadvantaged_PER_PROF,Not Economically Disadvantaged_TOTAL_SCALE_SCORES
0,1,2018,MATH3,NYC Public Schools,596,4327,23451,51704.0,45,30821973,NYC Public Schools,609,4593,12417,16969.0,73,10340003
1,1,2018,MATH4,NYC Public Schools,596,3811,20464,52157.0,39,31097878,NYC Public Schools,610,4339,11460,16686.0,69,10176883
2,1,2018,MATH5,NYC Public Schools,596,3577,18181,52128.0,35,31067028,NYC Public Schools,609,4423,10735,17246.0,62,10502124
3,1,2018,MATH6,NYC Public Schools,595,3105,17029,50798.0,34,30242711,NYC Public Schools,608,3530,9247,15157.0,61,9219483
4,1,2018,MATH7,NYC Public Schools,596,3204,16641,49938.0,33,29773842,NYC Public Schools,608,3471,9512,15776.0,60,9596865


# Merge Economic Dataset with All Students + Gender + Race + English Language Dataset

In [23]:
df_ela_2019_4 = pd.merge(
    df_ela_2019_3,
    df_annual_em_ela_2019_econ,
    on=["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],
    how="left",
)

df_ela_2019_4.head()


,ENTITY_CD,ENTITY_NAME,YEAR,ASSESSMENT_NAME,NOT_TESTED,NUM_TESTED,NUM_PROF,PER_PROF,TOTAL_SCALE_SCORES,MEAN_SCORE,...,Economically Disadvantaged_NUM_TESTED,Economically Disadvantaged_PER_PROF,Economically Disadvantaged_TOTAL_SCALE_SCORES,Not Economically Disadvantaged_ENTITY_NAME,Not Economically Disadvantaged_MEAN_SCORE,Not Economically Disadvantaged_NOT_TESTED,Not Economically Disadvantaged_NUM_PROF,Not Economically Disadvantaged_NUM_TESTED,Not Economically Disadvantaged_PER_PROF,Not Economically Disadvantaged_TOTAL_SCALE_SCORES
0,1,NYC Public Schools,2018,Combined8Math,21909,64087,27892,44,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NYC Public Schools,2018,Combined7Math,6675,65892,26320,40,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NYC Public Schools,2018,RegentsMath8,s,14361,11369,79,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,NYC Public Schools,2018,MATH3_8,60289,402824,177196,44,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,NYC Public Schools,2018,MATH8,21909,49726,16523,33,29864407,601,...,39213.0,30,23488521,NYC Public Schools,606,8984,4875,10513.0,46,6375886


# Dataframe for Special Education

In [24]:
df_annual_em_ela_2019_dis = df_annual_em_ela_2019[column_names].copy()

df_annual_em_ela_2019_dis = df_annual_em_ela_2019_dis[
    (df_annual_em_ela_2019_dis["SUBGROUP_NAME"] == "Students with Disabilities")
]


In [25]:
# reindex

df_annual_em_ela_2019_dis = df_annual_em_ela_2019_dis.set_index(
    ["SUBGROUP_NAME", "ENTITY_CD", "YEAR", "ASSESSMENT_NAME"]
).unstack(["SUBGROUP_NAME"])

df_annual_em_ela_2019_dis = df_annual_em_ela_2019_dis.reindex(
    columns=sorted(df_annual_em_ela_2019_dis.columns, key=lambda x: x[::-1])
)

df_annual_em_ela_2019_dis.columns = [
    "{}_{}".format(t, v) for v, t in df_annual_em_ela_2019_dis.columns
]

df_annual_em_ela_2019_dis = pd.DataFrame(df_annual_em_ela_2019_dis.to_records())


In [26]:
df_annual_em_ela_2019_dis.head()


,ENTITY_CD,YEAR,ASSESSMENT_NAME,Students with Disabilities_ENTITY_NAME,Students with Disabilities_MEAN_SCORE,Students with Disabilities_NOT_TESTED,Students with Disabilities_NUM_PROF,Students with Disabilities_NUM_TESTED,Students with Disabilities_PER_PROF,Students with Disabilities_TOTAL_SCALE_SCORES
0,1,2018,MATH3,NYC Public Schools,586,7088,3812,14523,26,8517178
1,1,2018,MATH4,NYC Public Schools,585,6565,2826,14987,19,8770719
2,1,2018,MATH5,NYC Public Schools,584,6381,2323,15083,15,8815958
3,1,2018,MATH6,NYC Public Schools,582,5286,1643,14255,12,8301798
4,1,2018,MATH7,NYC Public Schools,583,5124,1458,14039,10,8179488


# Merge Special Education Dataset with All Students + Gender + Race + English Language + Econ Dataset

In [27]:
df_ela_2019_5 = pd.merge(
    df_ela_2019_4,
    df_annual_em_ela_2019_dis,
    on=["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],
    how="left",
)

df_ela_2019_5.head()


,ENTITY_CD,ENTITY_NAME,YEAR,ASSESSMENT_NAME,NOT_TESTED,NUM_TESTED,NUM_PROF,PER_PROF,TOTAL_SCALE_SCORES,MEAN_SCORE,...,Not Economically Disadvantaged_NUM_TESTED,Not Economically Disadvantaged_PER_PROF,Not Economically Disadvantaged_TOTAL_SCALE_SCORES,Students with Disabilities_ENTITY_NAME,Students with Disabilities_MEAN_SCORE,Students with Disabilities_NOT_TESTED,Students with Disabilities_NUM_PROF,Students with Disabilities_NUM_TESTED,Students with Disabilities_PER_PROF,Students with Disabilities_TOTAL_SCALE_SCORES
0,1,NYC Public Schools,2018,Combined8Math,21909,64087,27892,44,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NYC Public Schools,2018,Combined7Math,6675,65892,26320,40,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NYC Public Schools,2018,RegentsMath8,s,14361,11369,79,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,NYC Public Schools,2018,MATH3_8,60289,402824,177196,44,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,NYC Public Schools,2018,MATH8,21909,49726,16523,33,29864407,601,...,10513.0,46,6375886,NYC Public Schools,585,6657,966,11503.0,08,6734679


# Clean final ELA dataset

In [28]:
# show all columns in dataframe

# Drop Entity Name columns

df_ela_2019_5 = df_ela_2019_5.drop(
    columns=['Female_ENTITY_NAME', 
    'Male_ENTITY_NAME',
    'American Indian or Alaska Native_ENTITY_NAME',
    'Black or African American_ENTITY_NAME',
    'Hispanic or Latino_ENTITY_NAME',
    'Asian or Native Hawaiian/Other Pacific Islander_ENTITY_NAME',
    'White_ENTITY_NAME',
    'Multiracial_ENTITY_NAME',
    'Non-English Language Learner_ENTITY_NAME',
    'English Language Learner_ENTITY_NAME',
    'Economically Disadvantaged_ENTITY_NAME',
    'Not Economically Disadvantaged_ENTITY_NAME',
    'Students with Disabilities_ENTITY_NAME']
)



In [29]:
df_ela_2019_5.head()

,ENTITY_CD,ENTITY_NAME,YEAR,ASSESSMENT_NAME,NOT_TESTED,NUM_TESTED,NUM_PROF,PER_PROF,TOTAL_SCALE_SCORES,MEAN_SCORE,...,Not Economically Disadvantaged_NUM_PROF,Not Economically Disadvantaged_NUM_TESTED,Not Economically Disadvantaged_PER_PROF,Not Economically Disadvantaged_TOTAL_SCALE_SCORES,Students with Disabilities_MEAN_SCORE,Students with Disabilities_NOT_TESTED,Students with Disabilities_NUM_PROF,Students with Disabilities_NUM_TESTED,Students with Disabilities_PER_PROF,Students with Disabilities_TOTAL_SCALE_SCORES
0,1,NYC Public Schools,2018,Combined8Math,21909,64087,27892,44,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NYC Public Schools,2018,Combined7Math,6675,65892,26320,40,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NYC Public Schools,2018,RegentsMath8,s,14361,11369,79,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,NYC Public Schools,2018,MATH3_8,60289,402824,177196,44,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,NYC Public Schools,2018,MATH8,21909,49726,16523,33,29864407,601,...,4875,10513.0,46,6375886,585,6657,966,11503.0,08,6734679


In [30]:
# save to csv

df_ela_2019_5.to_csv('df_ela_2019.csv', index=False)